<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import pandas as pd 
import numpy as np 
from wasabi import msg
from os import listdir
from tqdm import tqdm 
from os.path import isfile, join


pd.set_option('max_columns',100)
pd.set_option('max_row',5000)

In [80]:
# input parameters 

INPUT_DIR = './Datasets/orig/'
ONLY_MWE = False
WIKI_TITLE_DICT = '../Prodigy_anotation/allpages.txt' # wikipedia redirect page title
WIKI_TITLE_SEARCH = True
REMOVE_LABEL = ['university','person','researcher','country','location','scientist']
LIST_TO_CONCATE = './wikidump_terms.txt'

In [110]:
# read files 
files = [join(INPUT_DIR, f) for f in listdir(INPUT_DIR) if isfile(join(INPUT_DIR, f))]

remove_dfs = pd.DataFrame([], columns=['term', 'annotation', 'df'])
dfs = pd.DataFrame([], columns=['term', 'annotation', 'df'])

for file in files:
    msg.text(f"Preprocessing {file}...")
    df = pd.read_csv(file)
    
    
    if df.shape[1] != 3:
         raise ValueError(f"{file} is not in a good format. ['term', 'label', 'dataset']")
    
    # check annotation type and remove those that are not a term
    annotations = df.iloc[:,1].unique()
    to_remove = [ann for ann in annotations if ann in REMOVE_LABEL]
    
    if to_remove == []: 
        msg.good("There is no label to remove.")     
    else:
        for label in to_remove:
            msg.info(f"Removing terms with label {label} from {file}:")
            
            remove_df = df[df.annotation==label]
            df = df.drop(remove_df.index, axis=0)
            remove_dfs = remove_dfs.append(remove_df,sort=False)
             
            print('\n'.join(remove_df.term.values))
    
    # whether to keep only Multi-word expressions
    if ONLY_MWE:
        msg.info(f"Removing from single tokens from {file}:")
        
        single_tokens = [term for term in df.term.values if ' ' not in term]
        remove_df = df[df.term.isin(single_tokens)]
        df = df.drop(remove_df.index, axis=0)
        remove_dfs = remove_dfs.append(remove_df,sort=False)
        
        print('\n'.join(single_tokens))   
        
    dfs = dfs.append(df,sort=False)  
    print('\n')

Preprocessing ./Datasets/orig/annotation_df_scienceie.csv...
✔ There is no label to remove.


Preprocessing ./Datasets/orig/annotation_df_ncbi.csv...
✔ There is no label to remove.


Preprocessing ./Datasets/orig/annotation_df_scientific.csv...
ℹ Removing terms with label person from
./Datasets/orig/annotation_df_scientific.csv:
Brion James
Leon Kowalski
Joanna Cassidy
Zhora
Lee Sedol
Kiichiro Toyoda
Sakichi Toyoda
Mamie Van Doren
Pinky Lee
Harrison Ford
Rutger Hauer
Sean Young
Philip K. Dick
David Fincher
Kevin Spacey
Hampton Fancher
Sammon
Michael Deeley
Fancher
Ridley Scott
Manuel Toharia
Ragageles
Mick Foley
Rebecca Grant
Gary Hamel
C. K. Prahalad
Čapek
Samantha Ponder
Molly McGrath
Paul J. McAuley
Lester del Rey
Fernando Lamas
Arlene Dahl
David Arquette
Elizabeth Berkley
Thomas Jane
Norman McLaren
The Wachowskis
J. Michael Straczynski
Senator Edward M. Kennedy
Dalton
Daniel Pearl
Judea
John Waters
Walter R. Booth
Cecil Hepworth
Mustafa Suleyman
Chris Landreth
Zack Snyder
Clint Eas

In [111]:
if LIST_TO_CONCATE:
    with open(LIST_TO_CONCATE) as f:
        mwes = [x.strip() for x in f.readlines()]
    
    # remove duplicate
    mwes = list(set(mwes) - set(dfs.term.values))        
        
    df = pd.DataFrame(mwes, columns=['term'])
    dfs = dfs.append(df,sort=False).fillna(value={'annotation': 'TECH', 'df': 'wikidump'})

In [112]:
dfs 

,term,annotation,df
0,aperture z-scan experiments,Process,scienceie
1,1560nm femtosecond laser pulses,Material,scienceie
2,optical-chopper,Material,scienceie
3,vibrational combination states,Process,scienceie
4,non-radiative processes,Process,scienceie
...,...,...,...
716280,morning-after abortion pills,TECH,wikidump
716281,atcvet code qd09aa12,TECH,wikidump
716282,pac-man anniversary arcade machines,TECH,wikidump
716283,infiniti qx70/fx,TECH,wikidump


In [114]:
if WIKI_TITLE_DICT:
    wiki_title_dict = pd.read_csv(WIKI_TITLE_DICT, delimiter = "\t", header=None, index_col=0).to_dict()[1]
    dfs['wiki_title'] = dfs['term'].map(wiki_title_dict)

In [116]:
dfs.head(1000)

,term,annotation,df,wiki_title
0,aperture z-scan experiments,Process,scienceie,NaN
1,1560nm femtosecond laser pulses,Material,scienceie,NaN
2,optical-chopper,Material,scienceie,NaN
3,vibrational combination states,Process,scienceie,NaN
4,non-radiative processes,Process,scienceie,NaN
5,transient thermal effects,Process,scienceie,NaN
6,measure this cumulative thermal effect,Task,scienceie,NaN
7,mode-mismatched two-color pump–probe experiment,Process,scienceie,NaN
8,femtosecond laser pulses at 1560nm,Material,scienceie,NaN
9,pure optical nonlinearity,Process,scienceie,NaN


In [117]:
if WIKI_TITLE_SEARCH:
    # search for wiki page title that are not in the original dict
    import urllib 
    import requests 
    from bs4 import BeautifulSoup
    
    def find_wiki_title(term):
        encoded_term = urllib.parse.quote(term)
        
        url = f'https://en.wikipedia.org/w/api.php?action=query&redirects=true&titles={encoded_term}&format=json'
        json_response = requests.get(url).json()
        
        query = json_response.get('query')
        if query: 
            normalized = query.get('normalized')
            if normalized:
                redirect = normalized[0].get('to')
                return redirect

            
    # add wikititle for each term
    for term in tqdm(dfs[dfs.wiki_title.isna()].term.values[:10]):
        wiki_title = find_wiki_title(term)
        if wiki_title:
            wiki_title_dict.update({term:wiki_title})
            
            
    dfs['wiki_title'] = dfs['term'].map(wiki_title_dict)

100%|██████████| 10/10 [00:02<00:00,  4.46it/s]


In [12]:
# save data
dfs.to_csv (r'./matching_list.scv', index = False, header=True)
remove_dfs.to_csv (r'./removed_terms.scv', index = False, header=True)

In [ ]:
# 1. read all original csv 
# 2. check their types 
# 3. return type list 
# 4. whether to keep MWEs and remove single tokens 
# 5. if other list txt file, we can concatenate it 